# Multi Graph Agent Using LangGraph

In [13]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display

In [ ]:
groq_api_key = 'enter your api key'
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")

In [15]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str

# 1. Preprocessing Node
def preprocess(state: State) -> State:
    cleaned = state["messages"][-1].content.strip()
    state["messages"][-1].content = cleaned
    return state

# 2. Sentiment Analysis Node
def analyze_sentiment(state: State) -> State:
    msg = state["messages"][-1].content
    state["sentiment"] = "positive" if "good" in msg else "neutral"
    return state

# 3. Chatbot Node
def chatbot(state: State) -> State:
    return {"messages": llm.invoke(state['messages'])}

# 4. Logging Node
def logger(state: State) -> State:
    print(f"LOG: {state['messages'][-1].content}, Sentiment: {state.get('sentiment')}")
    return state

In [16]:
# Build the graph
builder = StateGraph(State)
builder.add_node("preprocess", preprocess)
builder.add_node("analyze_sentiment", analyze_sentiment)  # renamed
builder.add_node("chatbot", chatbot)
builder.add_node("logger", logger)

In [17]:
# Define flow
builder.add_edge(START, "preprocess")
builder.add_edge("preprocess", "analyze_sentiment")  # renamed
builder.add_edge("analyze_sentiment", "chatbot")     # renamed
builder.add_edge("chatbot", "logger")
builder.add_edge("logger", END)

In [18]:
graph = builder.compile()

In [19]:
# Draw the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [20]:
# Invoke the graph
input_message = "What are the Districts are there in Andraoradesh list that highetst economi?."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "What is the present population in india."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: Andhra Pradesh is a state in southern India, and it is divided into 26 districts. The economic performance of these districts can vary based on factors like industry, agriculture, infrastructure, and more. Here's a list of the top 5 districts in Andhra Pradesh by GDP (Gross Domestic Product) or economic performance:

1. **Visakhapatnam district**: Known for its port city, Visakhapatnam is a major industrial and economic hub in Andhra Pradesh. It has a strong presence of industries like steel, power, and pharmaceuticals.
2. **East Godavari district**: This district is a major contributor to the state's economy, with a strong focus on agriculture, aquaculture, and industrial sectors.
3. **Krishna district**: Located in the coastal region of Andhra Pradesh, Krishna district is a significant economic hub, with major industries like textiles, food processing, and pharmaceuticals.
4. **Guntur district**: Guntur is a major agricultural producer in the state, with a strong focus on crops 